In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [5]:
products = pd.read_csv('products_train.csv')
unique_locales = products['locale'].unique()

In [8]:
## LOCALES TO TRANSLATE
unique_locales = np.delete(unique_locales, np.where(unique_locales == 'UK'))
unique_locales

array(['DE', 'JP', 'ES', 'FR', 'IT'], dtype=object)

# German Machine Translation

reference: [English To German Model](https://huggingface.co/google/bert2bert_L-24_wmt_de_en)

## Sample example

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/bert2bert_L-24_wmt_de_en", pad_token="<pad>", eos_token="</s>", bos_token="<s>")
model = AutoModelForSeq2SeqLM.from_pretrained("google/bert2bert_L-24_wmt_de_en")

In [101]:
# Set the unk_token parameter
tokenizer.add_tokens(["[UNK]"])

model.encoder.resize_token_embeddings(len(tokenizer))

sentence = "Willst du einen Kaffee trinken gehen mit mir?"

input_ids = tokenizer(sentence, return_tensors="pt", add_special_tokens=False).input_ids
output_ids = model.generate(input_ids)[0]
print(tokenizer.decode(output_ids, skip_special_tokens=True))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (128) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Want to drink a kaffee go with me?


## Real fun part

In [89]:
def process(text):
    import re    # for regular expressions
    from nltk.stem import WordNetLemmatizer
    from nltk.tokenize import word_tokenize
    if isinstance(text, str):
        text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
        text = re.sub(r"[^\w\s]", "", text)
        # removing non-unicode characters
        # for example, it will remove character 'Ç'
        non_ascii_pattern = re.compile(r'[^\x00-\x7F]+')
        text = non_ascii_pattern.sub('', text)

        # Tokenize it
        text = tokenizer(text, truncation=True, padding=True)['input_ids']
        return text
    else:
        return ''

    # Return a string

In [96]:
## Get all the german rows
locale = 'DE'
de_products = products.loc[products['locale'] == locale]
de_products_title = products['title']

In [102]:
de_products_title.apply(process)

Exception: WordPiece error: Missing [UNK] token from the vocabulary

In [ ]:
print(de_products_title.head())
# output_ids = model.generate(de_products_title)